In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/soniame/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
from src import utils
conn = utils.connect_athena(path='../configs/athena.yaml')

In [3]:
from tabulate import tabulate

In [4]:
def extract_dictionary(database_name, table_name):
    df = pd.read_sql_query(f"""
        select * 
        from information_schema.columns
        where table_schema = '{database_name}' and table_name = '{table_name}';
        """, conn)
    
    return(df)

def write_dictionary_md(df, f):

    df['Description'] = ''
    
    print('\n', file=f)
    print('Database: ' + df.table_schema[1], file=f)
    print('Table: ' + df.table_name[1] + '\n', file=f)
    print(tabulate(df[['column_name', 'Description', 'data_type']].set_index('column_name'), 
                   tablefmt="pipe", headers="keys"), file=f)
    print('\n', file=f)

In [32]:
with open('../docs/Data Dictionary Internal.md', 'w') as f:
    write_dictionary_md(
        extract_dictionary(database_name = 'pwazetransformeddb', 
                           table_name = 'jams'), f)
    write_dictionary_md(
        extract_dictionary(database_name = 'spd_sdv_waze_corona', 
                           table_name = 'prod_daily_daily_index'), f)
    write_dictionary_md(
        extract_dictionary(database_name = 'spd_sdv_waze_corona', 
                           table_name = 'prod_daily_weekly_index'), f)
